In [2]:
import numpy as np
from scipy import stats
import pymesh
mesh = pymesh.load_mesh('bunny.obj')
#mesh = pymesh.load_mesh('square_2D.obj')
#mesh = pymesh.load_mesh('cube.obj')

mesh.enable_connectivity()

# print mesh.num_vertices, mesh.num_faces, mesh.num_voxels
# print mesh.dim, mesh.vertex_per_face, mesh.vertex_per_voxel
# print mesh.vertices


# Local feature calculation 



Lets try and calculate some features described in section 3.1.1 in https://arxiv.org/abs/1605.00129

First lets turn on some helpfull attributes in the mesh

In [16]:
mesh.add_attribute('vertex_normal')
mesh.add_attribute('vertex_laplacian')
mesh.add_attribute('vertex_mean_curvature')
mesh.add_attribute('vertex_gaussian_curvature')
mesh.add_attribute('vertex_index')

def feature_aggregate(x):
    ''' Feature aggregation function
        x: np.array
        returns: list of max,min,max-min,mean,var,harmonic mean
    '''
    return [x.max(),x.min(),x.max()-x.min(),x.mean(),x.var(),stats.hmean(x)]


## Euclidean distance

Calculate Euclidean distance the between neighborhood vertices to the tangent plane of a vertex v.


In [18]:
vi = 0 # Lets just choose the first for now

vertex_normal = mesh.get_attribute('vertex_normal').reshape(-1,mesh.dim)
neighbor_vertices_indexes = mesh.get_vertex_adjacent_vertices(vi)

# Get the target vertex index and normal  
n = vertex_normal[vi]
v = mesh.vertices[vi]

n_jk = vertex_normal[neighbor_vertices_indexes]
v_jk = mesh.vertices[neighbor_vertices_indexes]


#distances = np.abs((v - v_jk).dot(n))/np.linalg.norm(n)
distances = np.abs((v - v_jk).dot(n)) # n is already a unit vector
f_d = feature_aggregate(distances)

print 'distances: {}'.format(distances)
print 'f_d:       {}'.format(f_d)

distances: [0.00033088 0.00028909 0.00029419 0.00048537 0.00023373 0.00043732]
f_d:       [0.0004853726125383338, 0.00023372706719911146, 0.00025164554533922234, 0.00034509752390113313, 7.752714567834698e-09, 0.00032422060866673513]


## Angle θ of normal vector between v and v_kj

In [19]:
#thetas = np.arccos(np.dot(n_jk,n)/(np.linalg.norm(n)*np.linalg.norm(n_jk,axis=1)))
thetas = np.arccos(np.dot(n_jk,n)) # n and n_jk are already unit vectors 
f_theta = feature_aggregate(thetas)

print 'thetas(rad):   {}'.format(np.rad2deg(thetas))
print 'f_theta:       {}'.format(f_theta)


thetas(rad):   [11.901391    6.11664381  5.85407861  8.23079405  7.81086617 11.48873655]
f_theta:       [0.2077184584338905, 0.10217294649274043, 0.10554551194115007, 0.14952384113643233, 0.0017111207534338073, 0.13872063796282677]


## Gaussian, mean, and principal curvatures

http://mathworld.wolfram.com/PrincipalCurvatures.html


In [21]:
gaussian_curvature = mesh.get_attribute('vertex_gaussian_curvature')
mean_curvature = mesh.get_attribute('vertex_mean_curvature')

principal_curvature_1 = gaussian_curvature + (gaussian_curvature**2 - mean_curvature)**.5
principal_curvature_2 = gaussian_curvature - (gaussian_curvature**2 - mean_curvature)**.5

print 'c1:        {}'.format(principal_curvature_1[vi])
print 'c2:        {}'.format(principal_curvature_1[vi])
print 'c1c2:      {}'.format(gaussian_curvature[vi])
print '(c1+c2)/2: {}'.format(mean_curvature[vi])

f_c = [ principal_curvature_1[vi]
       ,principal_curvature_2[vi]
       ,mean_curvature[vi]
       ,gaussian_curvature[vi]]

print 'f_c:       {}'.format(f_c)

c1:        722.713539673
c2:        722.713539673
c1c2:      361.370915219
(c1+c2)/2: 20.4461183094
f_c:       [722.7135396728975, 0.028290764164523807, 20.44611830938878, 361.370915218531]
